# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-35.2705,174.0805,20.87,76,42,2.31,NZ,1702592950
1,1,gurupa,-1.4063,-51.6427,28.78,65,69,1.34,BR,1702592951
2,2,port alfred,-33.5982,26.8915,17.42,82,20,3.81,ZA,1702592952
3,3,kitui,-1.3666,38.0166,17.94,90,66,0.89,KE,1702592953
4,4,candido de abreu,-24.5747,-51.3432,25.27,75,45,0.81,BR,1702592460


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
#Configure the map plot
#YOUR CODE HERE

map_world1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo =True,
    alpha = 0.5,
    tiles = "OSM",
    color = "City",
    size = "Humidity",
    scale = 0.8,
    frame_width =700,
    frame_height = 500
)

# Display the map
# YOUR CODE HERE
map_world1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
nar_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] <= 27) & 
                               (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] <= 15)]
nar_city_df
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,papao,12.6919,-16.3792,22.06,78,0,0.00,SN,1702593099
139,139,torit,4.4134,32.5682,24.06,29,7,0.60,SS,1702593151
284,284,santana,-12.9792,-44.0506,25.45,19,1,0.92,BR,1702593308
303,303,guerrero negro,27.9710,-114.0364,22.48,31,0,3.97,MX,1702593327
328,328,ndele,4.9710,6.7413,26.12,81,14,1.55,NG,1702593349
334,334,posse,-14.0836,-46.3563,26.22,18,0,3.18,BR,1702593354
336,336,kawthoung,9.9781,98.5521,25.06,86,8,2.91,MM,1702593355
365,365,freetown,8.4790,-13.2680,26.02,74,1,2.80,SL,1702593384
416,416,bonthe,7.5273,-12.5020,25.98,74,0,1.23,SL,1702593437
423,423,malvan,16.0606,73.4669,22.89,49,0,3.05,IN,1702593445


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = nar_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
94,papao,SN,12.6919,-16.3792,78,
139,torit,SS,4.4134,32.5682,29,
284,santana,BR,-12.9792,-44.0506,19,
303,guerrero negro,MX,27.9710,-114.0364,31,
328,ndele,NG,4.9710,6.7413,81,
334,posse,BR,-14.0836,-46.3563,18,
336,kawthoung,MM,9.9781,98.5521,86,
365,freetown,SL,8.4790,-13.2680,74,
416,bonthe,SL,7.5273,-12.5020,74,
423,malvan,IN,16.0606,73.4669,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000

param = {
    # YOUR CODE HERE
    "categories":"accommodation.hotel",
    "limit":20,
    "filter":"",
    "bias":"", 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Long = row["Lng"]
    Lat = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    param["filter"] = f"circle:{Long},{Lat},{radius}"
    param["bias"] = f"proximity:{Long},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = param).json() # YOUR CODE HERE
    
    # Convert the API response to JSON format
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
papao - nearest hotel: No hotel found
torit - nearest hotel: Torit Hotel
santana - nearest hotel: No hotel found
guerrero negro - nearest hotel: San Ignacio
ndele - nearest hotel: No hotel found
posse - nearest hotel: Palmeiras Hotel
kawthoung - nearest hotel: Honey Bear
freetown - nearest hotel: High Peak Hotel
bonthe - nearest hotel: No hotel found
malvan - nearest hotel: Hotel Swastik Palace
barakani - nearest hotel: Hôtel Mang'Appart
heroica caborca - nearest hotel: Hotel Posada del Desierto
assab - nearest hotel: Ras Gambo Hotel
lagos - nearest hotel: LOBIM-D hotel
riachao das neves - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
94,papao,SN,12.6919,-16.3792,78,No hotel found
139,torit,SS,4.4134,32.5682,29,Torit Hotel
284,santana,BR,-12.9792,-44.0506,19,No hotel found
303,guerrero negro,MX,27.9710,-114.0364,31,San Ignacio
328,ndele,NG,4.9710,6.7413,81,No hotel found
334,posse,BR,-14.0836,-46.3563,18,Palmeiras Hotel
336,kawthoung,MM,9.9781,98.5521,86,Honey Bear
365,freetown,SL,8.4790,-13.2680,74,High Peak Hotel
416,bonthe,SL,7.5273,-12.5020,74,No hotel found
423,malvan,IN,16.0606,73.4669,49,Hotel Swastik Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_world2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo =True,
    alpha = 0.5,
    tiles = "OSM",
    color = "City",
    size = "Humidity",
    scale = 0.8,
    frame_width =700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
map_world2



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

{'type': 'FeatureCollection', 'features': []}